<a href="https://colab.research.google.com/github/jmamath/DFL-Contributions/blob/master/Variance_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math

# Global variable to indicate we are looking for 95 % confidence interval
Z_95 = 1.96


In [0]:
def naive_mc_simulation(n_sample, z=Z_95):
    """ 
    Example 1 - Naive Monte Carlo simulation in 
    "Simulation Efficiency and an Introduction to Variance Reduction Methods"    
    
    u and v follow a uniform distribution U(0,1) 
    we want to estimate the mean of y = exp{(u+w)^2}
    
    Args:
        n_sample: Int. Number of sample for the main simulation
        z: Float. constants to indicate the nature of the confidence interval 
    Returns:
        mean of Y
        half_width 
        confidence interval 
    """
    # random variable
    u = np.random.uniform(low=0, high=1, size=n_sample)
    w = np.random.uniform(low=0, high=1, size=n_sample)    
    y = np.exp(np.power(u+w,2))
    
    theta_n = np.mean(y)

    # Let's compute the half width confidence interval    
    half_width = z * np.std(y)/math.sqrt(n_sample)
    confidence_interval = (theta_n-half_width, theta_n+half_width)
    return theta_n, half_width, confidence_interval



In [0]:
def control_variate_MC_simulation(p_sample, n_sample, z_argument, Z=Z_95):
    """ 
    Example 1 - Control variate algorithm in 
    "Simulation Efficiency and an Introduction to Variance Reduction Methods"        
    
    u and v follow a uniform distribution U(0,1)   
    we want to estimate the mean of y = exp{(u+w)^2}
    
    Args:
        p_sample: Int. Number of sample for the pilot simulation
        n_sample: Int. Number of sample for the main simulation
        z_argument: Int. Number between 1 and 3 to choose the form of the control variate
                         random variable Z
        z: Float. constants to indicate the nature of the confidence interval 
    Returns:
        mean of Y
        half_width 
        confidence interval 
    """
    # (1) Pilot simulation 
    u = np.random.uniform(low=0, high=1, size=(p_sample))
    w = np.random.uniform(low=0, high=1, size=(p_sample))
    
    z = np.power(u+w,2)
    y = np.exp(z)
    
    cov = np.sum((z-np.mean(z))*(y-np.mean(y))) / p_sample-1
    c = -cov/np.var(z)
    
    # (2) Main simulation        
    u = np.random.uniform(low=0, high=1, size=(n_sample))
    w = np.random.uniform(low=0, high=1, size=(n_sample))
    
    z1 = u+w
    z2 = np.power(u+w,2)
    z3 = np.exp(u+w)
    
    if z_argument == 1:
        z = z1
    elif z_argument ==2:
        z = z2
    else:
        z = z3
    
    y = np.exp(np.power(u+w,2))    
    v = y + c*(z - np.mean(z))
    
    theta_c = np.mean(v)
    half_width = Z * np.std(v)/math.sqrt(n_sample)    
    confidence_interval = (theta_c-half_width, theta_c+half_width)  

    return theta_c, half_width, confidence_interval


In [0]:
param1, hw1, ci1 = naive_mc_simulation(1000)

param2, hw2, ci2 = control_variate_MC_simulation(100, 900,3)

naive = "Naive Monte Carlo \nMean estimation {} \nHalf Width: {}"
control_variate = "Control Variate Monte Carlo \nMean estimation {} \nHalf Width: {}"

print(naive.format(param1,hw1))
print("-----------------------")
print(control_variate.format(param2,hw2))


Naive Monte Carlo 
Mean estimation 4.429122366947787 
Half Width: 0.31980448298753467
-----------------------
Control Variate Monte Carlo 
Mean estimation 4.571490613929147 
Half Width: 0.1716633821310768
